# Ungraded Lab: Building a Custom Dense Layer

## Imports

In [1]:
import tensorflow as tf
import numpy as np

2024-07-16 12:58:51.219187: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 12:58:51.219344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 12:58:51.387040: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Custom Layer with weights

In [15]:
class SimpleDense(Layer):
    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = self.add_weight(name="kernel",
                                 shape=(input_shape[-1], self.units),
                                 initializer=w_init,
                                 trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = self.add_weight(name="bias",
                                 shape=(self.units,),
                                 initializer=b_init,
                                 trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b


In [16]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<KerasVariable shape=(1, 1), dtype=float32, path=simple_dense_6/kernel>, <KerasVariable shape=(1,), dtype=float32, path=simple_dense_6/bias>]


In [20]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# specify input shape when defining the Sequential model
model.build(input_shape=(None, 1))

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

# perform inference
print(model.predict(np.array([10.0])))

# see the updated state of the variables
print(my_layer.variables)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[[18.981577]]
[<KerasVariable shape=(1, 1), dtype=float32, path=sequential_5/simple_dense_11/kernel>, <KerasVariable shape=(1,), dtype=float32, path=sequential_5/simple_dense_11/bias>]
